# Preliminaries

Upload your kaggle API token. You can download it in the [kaggle account settings](https://www.kaggle.com/settings) under "Create new token"

In [ ]:
from google.colab import files
files.upload()
!chmod 600 kaggle.json
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

Set to `n` to limit the dataset to the first `n` entries or set to `0` if you want to use the entire dataset.

In [ ]:
limit_data = 1000

# Setup

## Download dependencies

In [ ]:
!pip install -q kaggle pyspark

In [ ]:
import pyspark

## Download and load dataset

In [ ]:
!kaggle datasets download yelp-dataset/yelp-dataset
!unzip -n yelp-dataset.zip

In [ ]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [ ]:
reviews = spark.read.json("yelp_academic_dataset_review.json")
if limit_data != 0: reviews = reviews.limit(limit_data)

In [ ]:
reviews.show()